In [ ]:
import pandas as pd
import talib
import numpy as np
from binance.client import Client

# Initialize the Binance client
api_key = 'api_key'
api_secret = 'your_secret_key'
client = Client(api_key, api_secret)

# Fetch historical data from Binance
def get_historical_klines(symbol, interval, lookback):
    klines = client.get_klines(symbol=symbol, interval=interval, limit=lookback)
    data = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 
                                         'close_time', 'quote_asset_volume', 'number_of_trades', 
                                         'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    data['close'] = data['close'].astype(float)
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    return data

# Fetch historical data for MATICUSDT with a 15-minute interval
symbol = 'ETHUSDT'
interval = '15m'
lookback = 1000  # You can increase this to get more data

df = get_historical_klines(symbol, interval, lookback)

# Calculate indicators
df['EMA200'] = talib.EMA(df['close'], timeperiod=200)
df['EMA50'] = talib.EMA(df['close'], timeperiod=50)
df['RSI'] = talib.RSI(df['close'], timeperiod=14)

# Initialize variables for backtesting
position = None
initial_balance = 10000  # Starting with $10,000
balance = initial_balance
qty = 4  # Number of units to trade
holdings = 0
entry_price = 0
trade_log = []

# Backtesting loop
for i in range(1, len(df)):
    current_price = df['close'].iloc[i]
    rsi = df['RSI'].iloc[i]
    timestamp = df['timestamp'].iloc[i]

    # Check if no position is currently held
    if position is None:
        # Long position entry condition
        if rsi > 70:
            position = 'LONG'
            entry_price = current_price
            holdings = qty
            balance -= entry_price * qty
            trade_log.append({'timestamp': timestamp, 'action': 'BUY', 'price': current_price, 'balance': balance, 'position': position})

        # Short position entry condition
        elif rsi < 30:
            position = 'SHORT'
            entry_price = current_price
            holdings = qty
            balance += entry_price * qty
            trade_log.append({'timestamp': timestamp, 'action': 'SELL', 'price': current_price, 'balance': balance, 'position': position})

    # If in a LONG position, check for exit condition
    elif position == 'LONG' and rsi < 70:
        balance += current_price * holdings
        holdings = 0
        trade_log.append({'timestamp': timestamp, 'action': 'SELL', 'price': current_price, 'balance': balance, 'position': 'EXIT'})
        position = None

    # If in a SHORT position, check for exit condition
    elif position == 'SHORT' and rsi > 30:
        balance -= current_price * holdings
        holdings = 0
        trade_log.append({'timestamp': timestamp, 'action': 'BUY', 'price': current_price, 'balance': balance, 'position': 'EXIT'})
        position = None

# Final balance calculation (accounting for any remaining positions)
if position == 'LONG':
    balance += current_price * holdings
elif position == 'SHORT':
    balance -= current_price * holdings

# Output results
print(f"Initial Balance: ${initial_balance}")
print(f"Final Balance: ${balance}")
print(f"Net Profit/Loss: ${balance - initial_balance}")

# Convert trade log to DataFrame for analysis
trade_log_df = pd.DataFrame(trade_log)

# Output the trade log
print(trade_log_df)

# Save trade log to CSV
trade_log_df.to_csv('trade_log.csv', index=False)

In [ ]:
import pandas as pd
import talib

# Step 1: Load the CSV and Inspect Columns
csv_file_path = ''  # Replace with your actual file path
df = pd.read_csv(csv_file_path)

# Map the 'open_time' column to 'timestamp' and convert it to datetime
df['timestamp'] = pd.to_datetime(df['open_time'], unit='ms')

# Ensure 'close' is of type float
df['close'] = df['close'].astype(float)

# Calculate indicators
df['EMA200'] = talib.EMA(df['close'], timeperiod=200)
df['EMA50'] = talib.EMA(df['close'], timeperiod=50)
df['RSI'] = talib.RSI(df['close'], timeperiod=14)



# Initialize variables for backtesting
position = None
initial_balance = 10000  # Starting with $10,000
balance = initial_balance
qty = 4  # Number of units to trade
holdings = 0
entry_price = 0
trade_log = []

print(df[['timestamp', 'close', 'RSI']])

# Backtesting loop
for i in range(1, len(df)):
    current_price = df['close'].iloc[i]
    rsi = df['RSI'].iloc[i]
    timestamp = df['timestamp'].iloc[i]

    # Check if no position is currently held
    if position is None:
        # Long position entry condition
        if rsi > 70:
            position = 'LONG'
            entry_price = current_price
            holdings = qty
            balance -= entry_price * qty
            trade_log.append({'timestamp': timestamp, 'action': 'BUY', 'price': current_price, 'balance': balance, 'position': position})

        # Short position entry condition
        elif rsi < 30:
            position = 'SHORT'
            entry_price = current_price
            holdings = qty
            balance += entry_price * qty
            trade_log.append({'timestamp': timestamp, 'action': 'SELL', 'price': current_price, 'balance': balance, 'position': position})

    # If in a LONG position, check for exit condition
    elif position == 'LONG' and rsi < 70:
        balance += current_price * holdings
        holdings = 0
        trade_log.append({'timestamp': timestamp, 'action': 'SELL', 'price': current_price, 'balance': balance, 'position': 'EXIT'})
        position = None

    # If in a SHORT position, check for exit condition
    elif position == 'SHORT' and rsi > 33:
        balance -= current_price * holdings
        holdings = 0
        trade_log.append({'timestamp': timestamp, 'action': 'BUY', 'price': current_price, 'balance': balance, 'position': 'EXIT'})
        position = None

# Final balance calculation (accounting for any remaining positions)
if position == 'LONG':
    balance += current_price * holdings
elif position == 'SHORT':
    balance -= current_price * holdings

# Output results
print(f"Initial Balance: ${initial_balance}")
print(f"Final Balance: ${balance}")
print(f"Net Profit/Loss: ${balance - initial_balance}")

# Convert trade log to DataFrame for analysis
trade_log_df = pd.DataFrame(trade_log)

# Output the trade log
print(trade_log_df)

# Save trade log to CSV
trade_log_df.to_csv('trade_log.csv', index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming the data is in a CSV file named 'trading_data.csv'
df = pd.read_csv('trade_log.csv')

# Filter the DataFrame to get only the rows where position is 'EXIT'
exit_positions = df[df['position'] == 'EXIT']

# Plot the balance amounts for the EXIT positions
plt.figure(figsize=(10, 6))
plt.plot(exit_positions['timestamp'], exit_positions['balance'], marker='o', linestyle='-', color='b')
plt.title('Balance Amount When Position is EXIT')
plt.xlabel('Timestamp')
plt.ylabel('Balance')
plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability
plt.grid(True)
plt.tight_layout()

# Show the plot
plt.show()